In [177]:
import tensorflow as tf
import numpy as np
import pandas as pd
from keras import layers
import keras

In [178]:
def training(len_input, embed_space_size, embed_action_size):
    '''define layer
    embed_space_size: space種類數
    embed_action_size: action種類數
    len_input: (space預計輸入維度, action預計輸入維度, others預計輸入維度)
    '''
    input_space = keras.Input(shape=len_input[0], name='Space_input')
    input_action = keras.Input(shape=len_input[1], name='Action_input')
    input_other = keras.Input(shape=len_input[2], name='Other_input')

    space_embedding = layers.Embedding(input_dim=embed_space_size, output_dim=4, mask_zero=True, name='Space_embedding')
    action_embeding = layers.Embedding(input_dim=embed_action_size, output_dim=4, mask_zero=True, name='Action_embedding')
    
    layer_reshape_space = layers.Reshape((4,), input_shape=(len_input[0], 4), name='reshape_space')
    layer_reshape_action = layers.Reshape((4,), input_shape=(len_input[1], 4), name='reshape_action')

    input_concat_embedding = layers.Concatenate(name='Input_merging', axis=1)
    layer_dense = layers.Dense(units=3, activation='softmax')

    '''define forward'''
    inputs = [input_space, input_action, input_other]
    embeded_space = space_embedding(input_space)
    embeded_action = action_embeding(input_action)

    embeded_space = layer_reshape_space(embeded_space)
    embeded_action = layer_reshape_action(embeded_action)

    embeded_input = input_concat_embedding([embeded_space, embeded_action, input_other])
    output_prob = layer_dense(embeded_input)

    model = keras.Model(inputs=inputs, outputs=output_prob, name='classifier')

    return model

---

prepare data

In [179]:
def prepare_data(df):
    input = df[['Team', 'No.', 'Space', 'Action']].copy()
    label = df[['Erros', 'Score']].copy()

    space = input[['Space']].astype('float')
    action = input[['Action']].astype('float')
    others = input[['Team', 'No.']].copy()

    others = others.astype({'No.': 'str'})
    others_dummy = pd.get_dummies(others).astype('float')

    label.loc[:, 'Nothing'] = 0.0
    label.loc[(label['Erros'] == 0) & (label['Score'] == 0), 'Nothing'] = 1.0

    space_tensor = tf.convert_to_tensor(space)
    action_tensor = tf.convert_to_tensor(action)
    others_tensor = tf.convert_to_tensor(others_dummy)
    
    x = [space_tensor, action_tensor, others_tensor]
    y = tf.convert_to_tensor(label)

    return x, y

In [180]:
train_df = pd.read_csv('train_df.csv')
test_df = pd.read_csv('test_df.csv')

In [181]:
train_x, train_y = prepare_data(train_df)
test_x, test_y = prepare_data(test_df)

In [200]:
embed_space_size = 15
embed_action_size = 9
model = training(
    (1, 1, train_x[2].shape[1]), 
    embed_space_size, 
    embed_action_size
    )
    
model.summary()

Model: "classifier"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Space_input (InputLayer)        [(None, 1)]          0                                            
__________________________________________________________________________________________________
Action_input (InputLayer)       [(None, 1)]          0                                            
__________________________________________________________________________________________________
Space_embedding (Embedding)     (None, 1, 4)         60          Space_input[0][0]                
__________________________________________________________________________________________________
Action_embedding (Embedding)    (None, 1, 4)         36          Action_input[0][0]               
_________________________________________________________________________________________

In [201]:
# regularizer = tf.keras.regularizers.l2(0.01)
optimizer = 'adam'
loss = keras.losses.CategoricalCrossentropy()
metrics = ['AUC']
epochs = 20
callbacks = tf.keras.callbacks.EarlyStopping(min_delta=0.002, patience=15, restore_best_weights=True)

model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [202]:
pred = model.fit(train_x, train_y, epochs=epochs, callbacks=[callbacks])

Epoch 1/20
29/29 [==============================] - 1s 2ms/step - loss: 1.1088 - auc: 0.5517
Epoch 2/20
29/29 [==============================] - 0s 2ms/step - loss: 1.0413 - auc: 0.6403
Epoch 3/20
29/29 [==============================] - 0s 3ms/step - loss: 0.9784 - auc: 0.7110
Epoch 4/20
29/29 [==============================] - 0s 2ms/step - loss: 0.9215 - auc: 0.7759
Epoch 5/20
29/29 [==============================] - 0s 2ms/step - loss: 0.8724 - auc: 0.8066
Epoch 6/20
29/29 [==============================] - 0s 3ms/step - loss: 0.8319 - auc: 0.8210
Epoch 7/20
29/29 [==============================] - 0s 3ms/step - loss: 0.7985 - auc: 0.8336
Epoch 8/20
29/29 [==============================] - 0s 2ms/step - loss: 0.7712 - auc: 0.8452
Epoch 9/20
29/29 [==============================] - 0s 2ms/step - loss: 0.7493 - auc: 0.8550
Epoch 10/20
29/29 [==============================] - 0s 2ms/step - loss: 0.7304 - auc: 0.8637
Epoch 11/20
29/29 [==============================] - 0s 2ms/step - lo